In [17]:
# 한국어 문장 처리 모델
# 챗봇의 유사도 계산
# https://github.com/kairess/mental-health-chatbot/blob/master/simple_chatbot.ipynb
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# SentenceBERT 모델 로드
# 구글에서 개발한 NLP(자연어처리) 훈련모델, 자연어 처리 분야에서 좋은 성능 Language Model
# SentenceTransformer에서 여러 모델중 1개를 사용
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
# 글자를 숫자벡터화 : 안녕하세요 숫자로 변함
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510458 -0.7733839   0.5927711  ...  0.57923526  0.3268346
  -0.65089655]
 [-0.09361681 -0.18191518 -0.19230832 ... -0.03165777  0.30412534
  -0.2679363 ]]


In [19]:
# 데이터 불러오기
df = pd.read_csv('wellness_dataset_original.csv')
df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


In [20]:
# 컬럼 Unnamed: 3 삭제
df = df.drop(columns=['Unnamed: 3'])
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [21]:
# isna() : 데이터 프레임 결측 값을 확인, 그 결과를 True, False로 반환
# 3,4의 NaN 데이터가 모두 삭제됨.
df = df[~df['챗봇'].isna()]
# df.to_csv('wellness_dataset_ok.csv') # 파일저장
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [22]:
# 0번째 row의 유저 컬럼 내용 출력
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [23]:
model.encode(df.loc[0, '유저'])

array([-4.80606616e-01, -2.94869423e-01,  4.37900215e-01, -6.40137851e-01,
        3.28670740e-02, -3.42647374e-01, -5.47481179e-02,  1.73056182e-02,
       -4.08221096e-01, -5.06034076e-01, -1.68733463e-01, -3.98676932e-01,
       -1.24776840e-01, -9.71538201e-02, -1.65286884e-01,  5.72612369e-03,
        6.13690987e-02, -1.91312388e-01,  2.53917605e-01, -5.85019708e-01,
       -2.84425557e-01, -2.32034907e-01, -3.27080548e-01,  6.72975630e-02,
       -1.62805827e-05, -4.72336292e-01, -3.60021859e-01,  2.91879863e-01,
       -6.63861156e-01, -3.10574383e-01,  5.79524994e-01, -3.11722606e-01,
        1.47696752e-02, -2.12172851e-01,  2.22057894e-01, -1.73828974e-01,
       -3.78458500e-01, -4.20398414e-01, -2.38218918e-01,  6.38703927e-02,
       -1.15304396e-01, -2.44563922e-01, -5.00228345e-01,  1.68355241e-01,
       -6.58360124e-01, -8.91941845e-01, -6.26956999e-01, -3.21965545e-01,
       -7.05358803e-01,  3.71447295e-01, -5.45803428e-01,  7.76295438e-02,
        1.09864593e-01,  

In [24]:
len(df)

1034

In [25]:

# 1034개 1차원 배열 생성
df['embedding'] = pd.Series([[]] * len(df)) # dummy
# 모델인코딩 후, 유저컬럼 글자들이 embedding컬럼에 벡터화, list저장
df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))
# df.to_csv('wellness_dataset_ok.csv') # embedding컬럼 추가해서 파일저장

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624..."


In [26]:
# 테스트 파일
text = '요즘 머리가 아프고 너무 힘들어'
# 모델링
embedding = model.encode(text)

In [27]:
# distance컬럼 추가 : 
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624...",0.315118


In [31]:
# 테스트 파일
text2 = '감정이 조절이 안되어요.'
# 모델링
embedding = model.encode(text2)

In [32]:
# distance컬럼 추가 : 
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
# df.to_csv('ChatBotData_em.csv')
df.head()

,구분,유저,챗봇,embedding,distance,similarity
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401...",0.448967,0.686329
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949...",0.490199,0.791652
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792...",0.352131,0.356176
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045...",0.422284,0.638450
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624...",0.315118,0.486297


In [33]:
answer = df.loc[df['similarity'].idxmax()]

print('구분 : ', answer['구분'])
print('유사한 질문 : ', answer['유저'])
print('챗봇 답변 : ', answer['챗봇'])
print('유사도 : ', answer['distance'])

구분 :  감정/감정조절이상
유사한 질문 :  더 이상 내 감정을 내가 컨트롤 못 하겠어.
챗봇 답변 :  저도 그 기분 이해해요. 많이 힘드시죠?
유사도 :  0.4901985824108124
